In [55]:
import imp
import utils.midi_support
imp.reload(utils.midi_support)
from utils.midi_support import MidiSupport


midi_path = "/Users/loreliegordon/Library/Mobile Documents/com~apple~CloudDocs/Documents/root/Columbia/Fall2021/ECBM4040/Project/rnn_music/music_generation_rnn/training_data/beethoven_opus10_1.mid"
mm = MidiSupport()
# mm.play_midi_file(midi_path)
midi_file = mm.load_midi_file(midi_path)

In [96]:
x_len = 6
y_len = 6

out = []

for i in range(x_len):
    line = []
    for j in range(y_len):
        if abs(i-j) < 3:
            line.append(1)
        else:
            line.append(0)
    out.append(line)

print(pd.DataFrame(out))


   0  1  2  3  4  5
0  1  1  1  0  0  0
1  1  1  1  1  0  0
2  1  1  1  1  1  0
3  0  1  1  1  1  1
4  0  0  1  1  1  1
5  0  0  0  1  1  1


In [57]:
import numpy as np
import string

def msg2dict(msg):
    result = dict()
    if 'note_on' in msg:
        on_ = True
    elif 'note_off' in msg:
        on_ = False
    else:
        on_ = None
    result['time'] = int(msg[msg.rfind('time'):].split(' ')[0].split('=')[1].translate(
        str.maketrans({a: None for a in string.punctuation})))

    if on_ is not None:
        for k in ['note', 'velocity']:
            result[k] = int(msg[msg.rfind(k):].split(' ')[0].split('=')[1].translate(
                str.maketrans({a: None for a in string.punctuation})))
    return [result, on_]

def switch_note(last_state, note, velocity, on_=True):
    # piano has 88 notes, corresponding to note id 21 to 108, any note out of this range will be ignored
    result = [0] * 88 if last_state is None else last_state.copy()
    if 21 <= note <= 108:
        result[note-21] = velocity if on_ else 0
    return result

def get_new_state(new_msg, last_state):
    new_msg, on_ = msg2dict(str(new_msg))
    new_state = switch_note(last_state, note=new_msg['note'], velocity=new_msg['velocity'], on_=on_) if on_ is not None else last_state
    return [new_state, new_msg['time']]

def track2seq(track):
    # piano has 88 notes, corresponding to note id 21 to 108, any note out of the id range will be ignored
    result = []
    last_state, last_time = get_new_state(str(track[0]), [0]*88)
    for i in range(1, len(track)):
        new_state, new_time = get_new_state(track[i], last_state)
        if new_time > 0:
            result += [last_state]*new_time
        last_state, last_time = new_state, new_time
    return result

def mid2arry(mid, min_msg_pct=0.1):
    tracks_len = [len(tr) for tr in mid.tracks]
    min_n_msg = max(tracks_len) * min_msg_pct
    # convert each track to nested list
    all_arys = []
    for i in range(len(mid.tracks)):
        if len(mid.tracks[i]) > min_n_msg:
            ary_i = track2seq(mid.tracks[i])
            all_arys.append(ary_i)
    # make all nested list the same length
    max_len = max([len(ary) for ary in all_arys])
    for i in range(len(all_arys)):
        if len(all_arys[i]) < max_len:
            all_arys[i] += [[0] * 88] * (max_len - len(all_arys[i]))
    all_arys = np.array(all_arys)
    all_arys = all_arys.max(axis=0)
    # trim: remove consecutive 0s in the beginning and at the end
    sums = all_arys.sum(axis=1)
    ends = np.where(sums > 0)[0]
    return all_arys[min(ends): max(ends)]

midi_np = mid2arry(midi_file)

In [98]:
midi_file.tracks[0]

MidiTrack([
  MetaMessage('track_name', name='Klaviersonate No. 5 -  1. Satz', time=0),
  MetaMessage('copyright', text='Copyright © 2008 by Bernd Krueger', time=0),
  MetaMessage('text', text='Ludwig van Beethoven', time=0),
  MetaMessage('text', text='Allegro molto e con brio', time=0),
  MetaMessage('text', text='Erstellt: 16.11.2008\n', time=0),
  MetaMessage('text', text='Update am 2.5.2012\n', time=0),
  MetaMessage('text', text='Dauer: 6:31 Minuten\n', time=0),
  MetaMessage('time_signature', numerator=3, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('key_signature', key='Eb', time=0),
  MetaMessage('set_tempo', tempo=327297, time=0),
  MetaMessage('marker', text='Teil1', time=0),
  MetaMessage('set_tempo', tempo=318302, time=960),
  MetaMessage('set_tempo', tempo=353753, time=1320),
  MetaMessage('set_tempo', tempo=327297, time=120),
  MetaMessage('set_tempo', tempo=351906, time=960),
  MetaMessage('set_tempo', tempo=333537, time=480)

In [99]:
import pandas as pd

pprint.pprint(midi_np[0:10, 10:35])

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 59,  0,  0, 59,  0,  0,  0, 59],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 59,  0,  0, 59,  0,  0,  0, 59],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 59,  0,  0, 59,  0,  0,  0, 59],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 59,  0,  0, 59,  0,  0,  0, 59],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 59,  0,  0, 59,  0,  0,  0, 59],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 59,  0,  0, 59,  0,  0,  0, 59],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 59,  0,  0, 59,  0,  0,  0, 59],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 59,  0,  0, 59,  0,  0,  0, 59],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 59,  0,  0, 59,  0,  0,  0, 59],
       [ 0,  0,  0,

In [103]:
midi_np

TypeError: _vhstack_dispatcher() takes 1 positional argument but 2 were given

In [77]:
lowerBound = 24
upperBound = 102
import itertools

def startSentinel():
    def noteSentinel(note):
        position = note
        part_position = [position]
        
        pitchclass = (note + lowerBound) % 12
        part_pitchclass = [int(i == pitchclass) for i in range(12)]
        
        return part_position + part_pitchclass + [0]*66 + [1] 
    return [noteSentinel(note) for note in range(upperBound-lowerBound)]

def getOrDefault(l, i, d):
    try:
        return l[i]
    except IndexError:
        return d

def buildContext(state):
    print(f"state is {state}")
    context = [0]*12
    for note, notestate in enumerate(state):
        if notestate[0] == 1:
            pitchclass = (note + lowerBound) % 12
            context[pitchclass] += 1
    return context
    
def buildBeat(time):
    return [2*x-1 for x in [time%2, (time//2)%2, (time//4)%2, (time//8)%2]]

def noteInputForm(note, state, context, beat):
    position = note
    part_position = [position]

    pitchclass = (note + lowerBound) % 12
    part_pitchclass = [int(i == pitchclass) for i in range(12)]
    # Concatenate the note states for the previous vicinity
    part_prev_vicinity = list(itertools.chain.from_iterable((getOrDefault(state, note+i, [0,0]) for i in range(-12, 13))))

    part_context = context[pitchclass:] + context[:pitchclass]

    return part_position + part_pitchclass + part_prev_vicinity + part_context + beat + [0]

def noteStateSingleToInputForm(state,time):
    beat = buildBeat(time)
    context = buildContext(state)
    return [noteInputForm(note, state, context, beat) for note in range(len(state))]

def noteStateMatrixToInputForm(statematrix):
    # NOTE: May have to transpose this or transform it in some way to make Theano like it
    #[startSentinel()] + 
    inputform = [ noteStateSingleToInputForm(state,time) for time,state in enumerate(statematrix) ]
    return inputform

noteStateMatrixToInputForm(midi_np[0:3])

state is [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 59  0  0 59  0  0  0 59  0  0  0  0 70  0  0 69  0  0  0 69  0  0
  0  0 82  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


IndexError: invalid index to scalar variable.

In [ ]:
from IPython.display import Audio
# from pretty_midi import PrettyMIDI

sf2_path = 'path/to/sf2'  # path to sound font file
midi_file = 'music.mid'

music = PrettyMIDI(midi_file=midi_file)
waveform = music.fluidsynth(sf2_path=sf2_path)
Audio(waveform, rate=44100)

In [ ]:
# Input X

# Input Size 50+12+4+12+1 = 79

# Position [1] MIDI value of current note # Why is the only 1? multiple notes can be played?
# Pitchclass [12]: 1 if current note and 0 else

# MIDI Note input range 0 to 127. Middle C C4 is 60

LSTM(127)



X = 

beat_input_X # (1,4) vector contain information of where in the 16 beat bar the current note is

In [9]:
def buildBeat(time):
    return [2*x-1 for x in [time%2, (time//2)%2, (time//4)%2, (time//8)%2]]

buildBeat(16)

[-1, -1, -1, -1]